## Importing libraries

In [1]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings
import ollama
import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent

warnings.filterwarnings("ignore")

## Using Ollama for accessing the model

In [2]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
    model="mistral",
    temperature=0,
)

### Defining the custom Tools

### Yahoo stock API

In [3]:
import yfinance as yf
def get_stock_price(ticker,history=5):
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    return df.to_string()

print(get_stock_price("TITAN"))

                  Close   Volume
Date                            
2024-11-05  3230.250000   872402
2024-11-06  3177.050049  4522923
2024-11-07  3120.850098  1608603
2024-11-08  3186.250000  1719882
2024-11-11  3206.600098  1054802


### Google search engine using BeautifulSoup

In [4]:
def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)

    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("Asian paints"))

Recent News:

0. Asian Paints shares plunge 9% as brokerages sound the alarm over 
underwhelming Q2 results
1. Asian Paints: Why this paints stock tumbled 9% today?
2. Asian Paints shares fall as much as 9%, most since April 2021, after 
downgrades, target cuts
3. Asian Paints shares nosedive 9% after Q2 PAT slips 42% YoY
4. Why Asian Paints Share Price is Falling



### Balance Sheets using yahoo finanace 

In [5]:
def get_financial_statements(ticker):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker+".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    
    return balance_sheet
print(get_financial_statements("TATAPOWER.NS"))

                                                                           2024-03-31       2023-03-31       2022-03-31
Ordinary Shares Number                                                   3195339547.0     3196991847.0     3196991847.0
Share Issued                                                             3195339547.0     3196991847.0     3196991847.0
Net Debt                                                               461555600000.0   447847600000.0   445128500000.0
Total Debt                                                             536895400000.0   529230900000.0   511952100000.0
Tangible Book Value                                                    291385400000.0   255477800000.0   192170700000.0
Invested Capital                                                       818351900000.0   777619500000.0   700316500000.0
Working Capital                                                       -127861500000.0  -130012100000.0  -141013900000.0
Net Tangible Assets                     

In [6]:
import requests
def get_company_Stock_ticker(query):
    url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={query}&apikey=E8HYQ13VA1F8KGTG'
    r = requests.get(url)
    data = r.json()

    company_name = data["bestMatches"][0]["2. name"]
    company_ticker = data["bestMatches"][0]["1. symbol"]
    return company_name,company_ticker

In [7]:
# Openai function calling

import json
import ollama
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_company_Stock_ticker",
                "description": "This will get the stock ticker of the company",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "this can be be a partial or full company name",
                        },
                    },
                    "required": ["query"],
                },
            },
        },
    ]

def get_stock_ticker(query):
    response = ollama.chat(
        model="mistral",
        messages=[
            {
                "role": "system",
                "content": "you are provided with the funciton get_company_Stock_ticker which retrieves the name and ticker of the requested company make sure to use the function",
            },
            {
                "role": "user",
                "content": f"what is the comapany name and the company stock ticker ?: {query}?",
            },
        ],
        tools=tools,
    )
    return get_company_Stock_ticker(response['message']['tool_calls'][0]["function"]["arguments"]["query"])


get_stock_ticker("get me stock symbol of Adani power")


('Adani Power Limited', 'ADANIPOWER.BSE')

In [8]:
def Anazlyze_stock(query):
    Company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"Company_name":Company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker,history=10)
    stock_financials=get_financial_statements(ticker)
    stock_news=get_recent_stock_news(Company_name)

    # available_information=f"Stock Price: {stock_data}\n\nStock Financials: {stock_financials}\n\nStock News: {stock_news}"
    available_information=f"Stock Financials: {stock_financials}\n\nStock News: {stock_news}"

    print("\n\nAnalyzing.....\n")
    analysis=llm(f"Give detail stock analysis, Use the available data and provide investment recommendation. \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {Company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} "
             )
    return analysis

In [9]:
print(Anazlyze_stock("Is it a good time to invest in Yes Bank?"))

{'Query': 'Is it a good time to invest in Yes Bank?', 'Company_name': 'Yes Bank Limited', 'Ticker': 'YESBANK.BSE'}


Analyzing.....

1. Financial Analysis:

   - The total assets of the company are $4,063,615,459, with cash and cash equivalents amounting to $6,340,570,100. This indicates a strong liquidity position.
   - The total liabilities of the company are $2,816,372,100, leaving a net assets value of $1,247,243,359.
   - The company's investments and advances are significant at $9,477,224,410, which could be a strategic move for future growth.
   - The company has reported a net PPE (Property, Plant, and Equipment) of $286,573,700 with an accumulated depreciation of $281,637,210, suggesting a moderate level of capital expenditure.

  2. Recent News:

   - The company has not disclosed any significant news in the provided timeframe. It is recommended to check the latest updates from reliable financial news sources or the company's official website for more recent information.

  3